## Polynomial representation

In [numpy](https://numpy.org/doc/stable) the concept of *array* is
generalized to imply arrays of arbitrary dimension, overlapping with the
concept of *scalars*, *matrices* and *tensors*. To allow arrays of
various dimensions to operate together it defines unambiguous
broadcasting rules for what to expect. The results is a library that is
used as the reference for almost all of the numerical Python community.

In mathematical literature the term *polynomial expansions* is used to
denote a collection of polynomials. Though they strictly do not need to,
they are often indexed, giving each polynomial both a label and a
position for where to locate a polynomial relative to the others.
Assuming that there always is an index, one could say that *polynomial
expansions* could just as well be termed *polynomial array*. And using
the rules defined in [numpy](https://numpy.org/doc/stable), there no
reason not to also start talking about *multi-dimensional polynomial
arrays*.

The main idea here is that in the same way as
`numpy.ndarray` are composed of scalars,
[chaospy.ndpoly](../../api/chaospy.ndpoly.rst) \-- the baseclass for
the polynomial arrays \-- are composed of simpler polynomials. This
gives us a mental model of a polynomial that looks like this:

$$\Phi(q_1, \dots, q_D) =
    [\Phi_1(q_1, \dots, q_D), \cdots, \Phi_N(q_1, \dots, q_D)]$$

where $\Phi$ is polynomial vector, $N$ is the number of terms in the
polynomial sum, and $q_d$ is the $d$-th indeterminant name. This mental
model is shown in practice in how chaospy represents its polynomials:

In [1]:
q0, q1 = chaospy.variable(2)
poly = chaospy.polynomial(4*q0+3*q1-1)
poly

polynomial(3*q1+4*q0-1)

Here [chaospy.variable()](../../api/chaospy.variable.rst) creates to simple
indeterminants, and the [chaospy.polynomial()](../../api/chaospy.polynomial.rst) constructor joins an array of polynomials into a polynomial
array, much like `numpy.array` does for
numbers in [numpy](https://numpy.org/doc/stable).

Another way to look at the polynomials is to keep the polynomial array
as a single polynomial sum: A multivariate polynomial can in the case of
`chaospy` be defined as:

$$\Phi(q_1, \dots, q_D) = \sum_{n=1}^N c_n q_1^{k_{1n}} \cdots q_D^{k_{Dn}}$$

where $c_n$ is a multi-dimensional polynomial coefficients, and $k_{nd}$
is the exponent for the $n$-th polynomial term and the $d$-th
indeterminant name.

Neither of the two ways of representing a polynomial array is incorrect,
and serves different purposes. The former works well for visualization,
while the latter form gives a better mental model of how `chaospy`
handles its polynomial internally.

Modeling polynomials by storing the coefficients as multi-dimensional
arrays is deliberate. Assuming few $k_{nd}$ and large dimensional $c_n$,
all numerical operations that are limited to the coefficients, can be
done fast, as [numpy](https://numpy.org/doc/stable) can do the heavy
lifting.

This way of representing a polynomial also means that to uniquely
defined a polynomial, we only need the three components:

-   [chaospy.ndpoly.coefficients](../../api/chaospy.ndpoly.coefficients.rst) \--
    the polynomial coefficients $c_n$ as multi-dimensional arrays.
-   [chaospy.ndpoly.exponents](../../api/chaospy.ndpoly.exponents.rst) \-- the
    exponents $k_{nd}$ as a 2-dimensional matrix.
-   [chaospy.ndpoly.indeterminants](../../api/chaospy.ndpoly.indeterminants.rst) \--
    the names of the variables, typically `q0`, `q1`, etc.

We can access these three defining properties directly from any
[chaospy.ndpoly](../../api/chaospy.ndpoly.rst) polynomial. For
example, for a simple polynomial:

In [2]:
poly.coefficients

[-1, 4, 3]

In [3]:
poly.exponents

array([[0, 0],
       [1, 0],
       [0, 1]], dtype=uint32)

In [4]:
poly.indeterminants

polynomial([q0, q1])

Because these three properties uniquely define a polynomial array, they
can also be used to reconstruct the original polynomial:

In [5]:
poly_ = poly.indeterminants**poly.exponents
poly_ = poly.coefficients*chaospy.prod(poly_, axis=-1)
poly_

polynomial([-1, 4*q0, 3*q1])

Here [chaospy.prod()](../../api/chaospy.prod.rst) and
[chaospy.sum()](../../api/chaospy.sum.rst) is used analogous to their
[numpy](https://numpy.org/doc/stable) counterparts
`numpy.prod` and
`numpy.sum` to multiply and add terms
together over an axis.

As mentioned the chosen representation works best with relatively few
$k_{nd}$ and large $c_n$. For large number $k_{nd}$ and relatively small
$c_n$ however, the advantage disappears. And even worse, in the case
where polynomial terms $q_1^{k_{1n}} \cdots
q_D^{k_{Dn}}$ are sparsely represented, the `chaospy` representation is
quite memory inefficient. So it is worth keeping in mind that the
advantage of this implementation depends a little upon what kind of
problems you are working on. It is not the tool for all problems.

### Polynomial expansions

As illustrated above, simple polynomial can be created through variable constructor
[chaospy.variable()](../../api/chaospy.variable.rst).
From these simpleton variables, any polynomials can be constructed using simple arithmetic and joining into expansions through [chaospy.polynomial()](../../api/chaospy.polynomial.rst):

In [6]:
chaospy.polynomial([1, q0, 1-q0*q1, q0**2*q1, q0-q1**2])

polynomial([1, q0, -q0*q1+1, q0**2*q1, -q1**2+q0])

In practice, having the ability to fine tune a polynomial exactly as one
wants it can be useful, but it can also be cumbersome when dealing with
larger arrays for application. To automate the construction of simple
polynomials, there is the [chaospy.monomial()](../../api/chaospy.monomial.rst) constructor. In its simplest forms it creates an array of
simple monomials:

In [7]:
chaospy.monomial(5)

polynomial([1, q0, q0**2, q0**3, q0**4])

It can be expanded to include number of dimensions and a lower bound for
the polynomial order:

In [8]:
chaospy.monomial(start=2, stop=3, dimensions=2)

polynomial([q0**2, q0*q1, q1**2])

Note that the polynomial is here truncated on total order, meaning that
the sum of exponents is limited by the $L_1$-norm. If a full
tensor-product of polynomials, or another norm is wanted in the
truncation, this is also possible using the `cross_truncation` flag:

In [9]:
chaospy.monomial(2, 3, dimensions=2, cross_truncation=numpy.inf)

polynomial([q0**2, q0**2*q1, q1**2, q0*q1**2, q0**2*q1**2])

In [10]:
chaospy.monomial(2, 4, dimensions=2, cross_truncation=0.8)

polynomial([q0**2, q0**3, q0*q1, q1**2, q1**3])

In [11]:
chaospy.monomial(2, 4, dimensions=2, cross_truncation=0.0)

polynomial([q0**2, q0**3, q1**2, q1**3])

Alternative to the [chaospy.monomial()](../../api/chaospy.monomial.rst)
function, it is also possible to achieve the same expansion using the
exponents only. For example:

In [12]:
q0**numpy.arange(5)

polynomial([1, q0, q0**2, q0**3, q0**4])

Or in the multivariate case:

In [13]:
expon = [[2, 0], [3, 0], [0, 2], [0, 3]]
base = chaospy.polynomial([q0, q1])
chaospy.prod(base**expon, axis=-1)

polynomial([q0**2, q0**3, q1**2, q1**3])

To help construct these exponent, there is function
[chaospy.glexindex()](../../api/chaospy.glexindex.rst). It behave the same
as [chaospy.monomial()](../../api/chaospy.monomial.rst), but only creates
the exponents. E.g.:

In [14]:
chaospy.glexindex(0, 5, 1).T

array([[0, 1, 2, 3, 4]])

In [15]:
chaospy.glexindex(2, 3, 2, numpy.inf).T

array([[2, 2, 0, 1, 2],
       [0, 1, 2, 2, 2]])

In [16]:
chaospy.glexindex(2, 4, 2, 0.8).T

array([[2, 3, 1, 0, 0],
       [0, 0, 1, 2, 3]])

In [17]:
chaospy.glexindex(2, 4, 2, 0.0).T

array([[2, 3, 0, 0],
       [0, 0, 2, 3]])

### Polynomial evaluation

Polynomials are not polynomials if they can not be evaluated as such. In
the case of `chaospy`, this can be done using object call. `chaospy`
supports calls with both positional arguments and by name. In other
words, one argument per variable:

In [18]:
expansion = chaospy.polynomial([1, q0**2, q0*q1])
expansion

polynomial([1, q0**2, q0*q1])

In [19]:
expansion(2, 1)

array([1, 4, 2])

In [20]:
expansion(q0=2, q1=1)

array([1, 4, 2])

Here the return value is a `numpy.ndarray`. However, it is also possible to get a polynomial in
return, given a partial evaluations:

In [21]:
expansion(3)

polynomial([1, 9, 3*q1])

In [22]:
expansion(q0=3)

polynomial([1, 9, 3*q1])

For positional evaluation, to allow for partial evaluations of variables
beyond the first, it is possible to pass a `None` value to the
polynomial to indicate that a variable is not to be touched in a partial
evaluation. E.g.:

In [23]:
expansion(None, 2)

polynomial([1, q0**2, 2*q0])

In [24]:
expansion(q1=2)

polynomial([1, q0**2, 2*q0])

Vectorized evaluations is also allowed. Just pass any
`numpy.ndarray` compatible object.
`chaospy` will expand the shape such that it ends up being
`polynomial.shape+input.shape`. For example:

In [25]:
expansion(q1=range(4))

polynomial([[1, 1, 1, 1],
            [q0**2, q0**2, q0**2, q0**2],
            [0, q0, 2*q0, 3*q0]])

It is also possible to mix both scalar and vector arguments, as long as
they are broadcastable in [numpy](https://numpy.org/doc/stable) sense.
For example:

In [26]:
expansion(1, [1, 2, 3])

array([[1, 1, 1],
       [1, 1, 1],
       [1, 2, 3]])

In [27]:
expansion([1, 2, 3], [1, 2, 3])

array([[1, 1, 1],
       [1, 4, 9],
       [1, 4, 9]])

Passing arguments with an other datatype than the polynomial, results in
the output using the common denomination:

In [28]:
expansion(0.5)

polynomial([1.0, 0.25, 0.5*q1])

In [29]:
expansion(q1=1j)

polynomial([(1+0j), q0**2, 1j*q0])

Assuming the input you want to evaluate is a large matrix and you want
an interface where the matrix is kept intact, you can use
[chaospy.call()](../../api/chaospy.call.rst). E.g.:

In [30]:
array = numpy.arange(12).reshape(2, 6)
array

array([[ 0,  1,  2,  3,  4,  5],
       [ 6,  7,  8,  9, 10, 11]])

In [31]:
chaospy.call(expansion, array)

array([[ 1,  1,  1,  1,  1,  1],
       [ 0,  1,  4,  9, 16, 25],
       [ 0,  7, 16, 27, 40, 55]])

Lastly, it is also possible to pass other polynomials as arguments. This
simplifies any form for variable substitution.

In [32]:
expansion

polynomial([1, q0**2, q0*q1])

In [33]:
expansion(q0=q1, q1=q0)

polynomial([1, q1**2, q0*q1])

In [34]:
expansion(None, 1-q1**3)

polynomial([1, q0**2, -q0*q1**3+q0])

### Numpy functions

The `chaospy` concept of arrays is taken from
[numpy](https://numpy.org/doc/stable). But it goes a bit deeper than
just inspiration. The base class [chaospy.ndpoly](../../api/chaospy.ndpoly.rst) is a direct subclass of `numpy.ndarray`:

In [35]:
issubclass(chaospy.ndpoly, numpy.ndarray)

True

The intentions is to have a library that is fast with the respect of the
number of coefficients, as it leverages
[numpy](https://numpy.org/doc/stable)\'s speed where possible.

In addition `chaospy` is designed to be behave both as you would expect
as a polynomial, but also, where possible, to behave as a
[numpy](https://numpy.org/doc/stable) numerical array. In practice this
means that `chaospy` provides a lot functions that also exists in
[numpy](https://numpy.org/doc/stable), which does about the same thing.
If one of these `chaospy` function is provided with a
`numpy.ndarray` object, the returned
values is the same as if provided to the
[numpy](https://numpy.org/doc/stable) function with the same name. For
example [chaospy.transpose()](../../api/chaospy.transpose.rst):

In [36]:
num_array = numpy.array([[1, 2], [3, 4]])
chaospy.transpose(num_array)

polynomial([[1, 3],
            [2, 4]])

And this works the other way around as well. If a polynomial is provided
to the [numpy](https://numpy.org/doc/stable) function, it will behave
the same way as if it was provided to the `chaospy` equivalent. So
following the same example, we can use
`numpy.transpose` to transpose
[chaospy.ndpoly](../../api/chaospy.ndpoly.rst) polynomials:

In [37]:
poly_array = chaospy.polynomial([[1, q0-1], [q1**2, 4]])
numpy.transpose(poly_array)

polynomial([[1, q1**2],
            [q0-1, 4]])

Though the overlap in functionality between
[numpy](https://numpy.org/doc/stable) and `chaospy` is large, there are
still lots of functionality which is specific for each of them. The most
obvious, in the case of `chaospy` features not found in
[numpy](https://numpy.org/doc/stable) is the ability to evaluate the
polynomials:

In [38]:
poly = q1**2-q0
poly

polynomial(q1**2-q0)

In [39]:
poly(4, 4)

12

In [40]:
poly(4)

polynomial(q1**2-4)

In [41]:
poly([1, 2, 3])

polynomial([q1**2-1, q1**2-2, q1**2-3])

### Function compatibility

The numpy library comes with a large number of functions for
manipulating `numpy.ndarray` objects.
Many of these functions are supported `chaospy` as well.
In addition [numpy](https://numpy.org/doc/stable)
provides dispatching of its functions to subclasses. This
means that functions in `chaospy` with the same name as a numpy
counterpart, it will work the same irrespective if the function used
was from [numpy](https://numpy.org/doc/stable) or `chaospy`, as the
former will pass any job to the latter.

For example:

In [42]:
chaospy.sum(expansion, keepdims=True)

polynomial([q0*q1+q0**2+1])

In [43]:
numpy.sum(expansion, keepdims=True)

polynomial([q0*q1+q0**2+1])

In addition, not everything is possible to support, and even within the
list of supported functions, not all use cases can be covered. Bit if
such an unsupported edge case is encountered, an
[chaospy.FeatureNotSupported](../../api/chaospy.FeatureNotSupported.rst) error
should be raised, so it should be obvious when they happen.

As a developer note, `chaospy` aims at being backwards compatible with
[numpy](https://numpy.org/doc/stable) as far as possible when it comes
to the functions it provides. This means that all functions below should
as far as possible mirror the behavior their
[numpy](https://numpy.org/doc/stable) counterparts, and for polynomial
constant, they should be identical (except for the object type).
Function that provides behavior not covered by
[numpy](https://numpy.org/doc/stable) should be placed elsewhere.

### Comparison operators

Because (real) numbers have a natural total ordering, mathematically
speaking, doing comparisons and sorting is at least conceptually for the
most part trivial. There are a few exceptions though. Take for example
complex numbers, which does not have a total ordering, it then there are
not always possible to assess if one number is large than the other. To
get around this limitation, some choices has to be made. For example, in
pure Python the choice to raise exception for all comparison of complex
numbers:

In [44]:
from pytest import raises

with raises(TypeError):
    1+3j > 3+1j

In `numpy` a different choice were made. Comparisons of complex numbers
are supported, but they limit the compare to the real part only,
ignoring the imaginary part of the numbers. For example:

In [45]:
arr = numpy.array([1+1j, 1+3j, 3+1j, 3+3j])
arr > arr[::-1]

array([False, False,  True,  True])

Polynomials does not have a total ordering either, and imposing one
requires many choices dealing with various edge cases. However, it is
possible to impose a total order that is both internally consistent and
which is backwards compatible with the behavior of `numpy.ndarray`. It
requires some design choices, which are opinionated, and might not
always align with everyone's taste.
With this in mind, the ordering implemented in `chaospy` is defined as
follows.

Polynomials containing terms with the highest exponents are
considered the largest:

In [46]:
q0 < q0**2 < q0**3

True

If the largest polynomial exponent in one polynomial is larger than
in another, leading coefficients are ignored:

In [47]:
4*q0 < 3*q0**2 < 2*q0**3

True

In the multivariate case, the polynomial order is determined by the
sum of the exponents across the indeterminants that are multiplied
together:

In [48]:
q0**2*q1**2 < q0*q1**5 < q0**6*q1

True

This implies that given a higher polynomial order, indeterminant names are ignored:

In [49]:
q0, q1, q2 = chaospy.variable(3)
q0 < q2**2 < q1**3

True

The same goes for any polynomial terms which are not leading:

In [50]:
4*q0 < q0**2+3*q0 < q0**3+2*q0

True

Here leading means the term in the polynomial that is the largest,
as defined by the rules here so far.
                                                                   
Polynomials of equal polynomial order are reverse
lexicographical sorted:

In [51]:
q0 < q1 < q2

True

As with polynomial order, coefficients and lower order terms are
also ignored:

In [52]:
4*q0**3+4*q0 < 3*q1**3+3*q1 < 2*q2**3+2*q2

True

Composite polynomials of the same order are lexicographical sorted
by the dominant indeterminant name:

In [53]:
q0**3*q1 < q0**2*q1**2 < q0*q1**3

True

If there are more than two indeterminants, the dominant order first
addresses the first name (lexicographical sorted), then the
second, and so on:

In [54]:
q0**2*q1**2*q2 < q0**2*q1*q2**2 < q0*q1**2*q2**2

True

Polynomials that have the same leading polynomial exponents, are
compared by the leading polynomial coefficient:

In [55]:
-4*q0 < -1*q0 < 2*q0

True

This notion implies that constant polynomials behave in the same way
as `numpy` arrays:

In [56]:
chaospy.polynomial([2, 4, 6]) > 3

array([False,  True,  True])

Polynomials with the same leading polynomial and coefficient are
compared on the next largest leading polynomial:

In [57]:
q0**2+1 < q0**2+2 < q0**2+3

True

And if both the first two leading terms are the same, use the third
and so on:

In [58]:
q0**2+q0+1 < q0**2+q0+2 < q0**2+q0+3

True

Unlike for the leading polynomial term, missing terms are considered
present as 0. E.g.:

In [59]:
q0**2-1 < q0**2 < q0**2+1

True

These rules together allow for a total comparison for all polynomials.

In `chaospy`, there are a few global options that can be passed to
[chaospy.set_options()](../../api/chaospy.set_options.rst) (or
[chaospy.global_options()](../../api/chaospy.global_options.rst)) to change this
behavior. In particular:

-  `sort_graded`

    Impose that polynomials are sorted by grade, meaning the indices are
    always sorted by the index sum. E.g. `q0**2*q1**2*q2**2` has an
    exponent sum of 6, and will therefore be consider larger than both
    `q0**3*q1*q2`, `q0*q1**3*q2` and `q0*q1*q2**3`. Defaults to true.

-   `sort_reverse`

    Impose that polynomials are sorted by reverses lexicographical
    sorting, meaning that `q0*q1**3` is considered smaller than
    `q0**3*q1`, instead of the opposite. Defaults to false.

### Polynomial division

Numerical division can be split into two variants: floor division
([chaospy.floor_divide()](../../api/chaospy.floor_divide.rst)) and true
division ([chaospy.true_divide()](../../api/chaospy.true_divide.rst)):

In [60]:
dividend = 7
divisor = 2
quotient_true = numpy.true_divide(dividend, divisor)
quotient_true

3.5

In [61]:
quotient_floor = numpy.floor_divide(dividend, divisor)
quotient_floor

3

The discrepancy between the two can be captured by a remainder
([chaospy.remainder()](../../api/chaospy.remainder.rst)), which allow us to
more formally define them as follows:

In [62]:
remainder = numpy.remainder(dividend, divisor)
remainder

1

In [63]:
dividend == quotient_floor*divisor+remainder

True

In [64]:
dividend == quotient_true*divisor

True

In the case of polynomials, neither true nor floor division is supported
like this. Instead it support its own kind of polynomial division
[chaospy.poly_divide()](../../api/chaospy.poly_divide.rst). Polynomial
division falls back to behave like floor division for all constants, as
it does not round values:

In [65]:
dividend = q0**2+q1
divisor = q0-1
quotient = chaospy.poly_divide(dividend, divisor)
quotient

polynomial(q0+1.0)

However, like floor division, it can still have remainders using
[chaospy.poly_remainder()](../../api/chaospy.poly_remainder.rst). For example:

In [66]:
remainder = chaospy.poly_remainder(dividend, divisor)
remainder

polynomial(q1+1.0)

In [67]:
dividend == quotient*divisor+remainder

True

In `numpy`, the \"Python syntactic sugar\" operators have the following
behavior:

-   `/` is used for true division
    [chaospy.true_divide()](../../api/chaospy.true_divide.rst).
-   `//` is used for floor division
    [chaospy.floor_divide()](../../api/chaospy.floor_divide.rst).
-   `%` is used for remainder [chaospy.remainder()](../../api/chaospy.remainder.rst).
-   `divmod` is used for floor division and remainder in combination to
    save computational cost through [chaospy.divmod()](../../api/chaospy.divmod.rst).

In `chaospy`, which takes precedence if any of the values are of
[chaospy.ndpoly](../../api/chaospy.ndpoly.rst) objects, take the
following behavior:

-   `/` is used for polynomial division
    [chaospy.poly_divide()](../../api/chaospy.poly_divide.rst), which is not
    compatible with [numpy](https://numpy.org/doc/stable).
-   `//` is still used for floor division
    [chaospy.floor_divide()](../../api/chaospy.floor_divide.rst) which is
    compatible with [numpy](https://numpy.org/doc/stable), and only
    works if divisor is a constant.
-   `%` is used for polynomial remainder
    [chaospy.poly_remainder()](../../api/chaospy.poly_remainder.rst), which is
    not backwards compatible.
-   `divmod` uses [chaospy.poly_divmod()](../../api/chaospy.poly_divmod.rst)
    which is used to save computation cost by doing
    [chaospy.poly_divide()](../../api/chaospy.poly_divide.rst) and
    [chaospy.remainder()](../../api/chaospy.remainder.rst) at the same time.